# Service Account Interactive Login

**Service accounts (svc_*) should not perform interactive logons (4624 LogonType 2/10).**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **logon type**, **source host**, **privileges**, **recent password/secret changes**.


## Investigation Queries

Identify interactive logons by service accounts and pivot to privileged group membership.


In [ ]:
// KQL
SecurityEvent
| where TimeGenerated > ago(7d)
| where EventID == 4624
| where Account hasprefix 'svc_' or Account hasprefix 'service_' or Account endswith '_svc'
| where LogonType in (2,10)
| project TimeGenerated, Computer, Account, IpAddress, LogonType


In [ ]:
# Splunk SPL
index=auth sourcetype=WinEventLog:Security EventCode=4624
(Account_Name="svc_*" OR Account_Name="service_*" OR Account_Name="*_svc") Logon_Type IN (2,10)
| table _time host Account_Name Logon_Type src_ip


In [ ]:
# Elastic (KQL/EQL)
winlog.event_id:4624 AND (user.name:svc_* OR user.name:service_* OR user.name:*_svc) AND winlog.event_data.LogonType:(2 OR 10)


## Containment & Response

Disable interactive logon capability, rotate secrets, and investigate lateral movement.


In [ ]:
# PowerShell
## Disable account (AD example)
Disable-ADAccount -Identity <SVC_ACCOUNT>

## Rotate secret in your secrets manager (process-specific)
# Update service principal / application secret


In [ ]:
# PowerShell
## Deny interactive logon via GPO is recommended.
# Validate 'Deny log on locally' / 'Deny log on through Remote Desktop Services'
